In [ ]:


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# Any results you write to the current directory are saved as output.
from sklearn.ensemble import  RandomForestClassifier as RFC
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from sklearn.metrics import f1_score,accuracy_score ,roc_auc_score
from nltk.stem.snowball import SnowballStemmer
import xgboost as xgb
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LogisticRegression as LR

In [ ]:
# df = pd.read_csv("/kaggle/input/yelp-reviews-dataset/yelp.csv")
df = pd.read_csv("/kaggle/input/redditnew/reddit.csv")
df.head()

In [ ]:
# # df = df[["text","funny"]]
# li = df["funny"].values
# op =[]
# for i in li :
#     if int(i) >2 :
#             op.append(1)
#     else :
#             op.append(0)
# df["funny"] = op
# df.columns = ["tweet","Sarcasm"]

In [ ]:
df.head()

In [ ]:
df= df.dropna()
df = df[df.tweet.apply(lambda x: x !="")]

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
df.dropna(inplace =True)
posi =[]
negi =[]
neuu =[]
compi =[]
for i in df.tweet.values:
    score = analyser.polarity_scores(str(i))
    score = dict(score)
    posi.append(score['pos'])
    negi.append(score['neg'])
    neuu.append(score['neu'])
    compi.append(score['compound'])
    

In [ ]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [ ]:
# for i in df["tweet"].values:
#     print(i)

In [ ]:
gb =df
gb = gb["tweet"]
modified =[]
for i in gb.values :
    url = re.sub('\#sarcasm$', '', i)
    modified.append(url)
df["tweet"] = modified


In [ ]:
# for i in df["tweet"].values:
#     print(i)

In [ ]:
df["tweet"] = df["tweet"].map(lambda x: clean_text(x))

In [ ]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df["tweet"])
sequences = tokenizer.texts_to_sequences(df["tweet"])
data = pad_sequences(sequences, maxlen=50)

**Extract word embeddings from the Glove**

In [ ]:
embeddings_index = dict()
f = open('/kaggle/input/glove/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

**Create a weight matrix**

In [ ]:
embedding_matrix = np.zeros((vocabulary_size, 50))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
import tensorflow as tf
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
import keras
config = tf.ConfigProto( device_count = {'GPU': 1 } ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [ ]:
labels = df["Sarcasm"]

In [ ]:
data.shape


In [ ]:
for i in range (0, (10)):
    for j in range (0, len(data[i])):
        print(data[i][j] , end = " ")
    print("")



In [ ]:
 from sklearn.decomposition import PCA
pca = PCA(n_components=2)
frosty = pca.fit_transform(data)

In [ ]:
for i in range (0, (10)):
    for j in range (0, len(frosty[i])):
        print(frosty[i][j] , end = " ")
    print("")



In [ ]:
# import numpy as np
# from numpy.linalg import norm


# class Kmeans:
#     '''Implementing Kmeans algorithm.'''

#     def __init__(self, n_clusters, max_iter=100, random_state=123):
#         self.n_clusters = n_clusters
#         self.max_iter = max_iter
#         self.random_state = random_state

#     def initializ_centroids(self, X):
#         np.random.RandomState(self.random_state)
#         random_idx = np.random.permutation(X.shape[0])
#         centroids = X[random_idx[:self.n_clusters]]
#         return centroids

#     def compute_centroids(self, X, labels):
#         centroids = np.zeros((self.n_clusters, X.shape[1]))
#         for k in range(self.n_clusters):
#             centroids[k, :] = np.mean(X[labels == k, :], axis=0)
#         return centroids

#     def compute_distance(self, X, centroids):
#         distance = np.zeros((X.shape[0], self.n_clusters))
#         for k in range(self.n_clusters):
#             row_norm = norm(X - centroids[k, :], axis=1)
#             distance[:, k] = np.square(row_norm)
#         return distance

#     def find_closest_cluster(self, distance):
#         return np.argmin(distance, axis=1)

#     def compute_sse(self, X, labels, centroids):
#         distance = np.zeros(X.shape[0])
#         for k in range(self.n_clusters):
#             distance[labels == k] = norm(X[labels == k] - centroids[k], axis=1)
#         return np.sum(np.square(distance))
    
#     def fit(self, X):
#         self.centroids = self.initializ_centroids(X)
#         for i in range(self.max_iter):
#             old_centroids = self.centroids
#             distance = self.compute_distance(X, old_centroids)
#             self.labels = self.find_closest_cluster(distance)
#             self.centroids = self.compute_centroids(X, self.labels)
#             if np.all(old_centroids == self.centroids):
#                 break
#         self.error = self.compute_sse(X, self.labels, self.centroids)
    
#     def predict(self, X):
#         distance = self.compute_distance(X, old_centroids)
#         return self.find_closest_cluster(distance)

In [ ]:
# newfrosty = frosty[ 0:100 , 0:2]

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import MinMaxScaler
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline
# km = Kmeans(n_clusters=2, max_iter=5)
# km.fit(newfrosty)
# centroids = km.centroids


In [ ]:

# # Plot the clustered data
# fig, ax = plt.subplots(figsize=(6, 6))
# plt.scatter(newfrosty[km.labels == 0], newfrosty[km.labels == 0],
#             c='green', label='cluster 1')
# plt.scatter(newfrosty[km.labels == 1], newfrosty[km.labels == 1],
#             c='blue', label='cluster 2')
# plt.scatter(centroids[:, 0], centroids[:, 0], marker='*', s=300,
#             c='r', label='centroid')
# plt.legend()
# plt.xlim([-2, 2])
# plt.ylim([-2, 2])
# plt.xlabel('Eruption time in mins')
# plt.ylabel('Waiting time to next eruption')
# plt.title('Visualization of clustered data', fontweight='bold')
# ax.set_aspect('equal')


In [ ]:
# for i in range (0, (100)):
#     for j in range (0, len(embedding_matrix[i])):
#         print(embedding_matrix[i][j] , end = " ")
#     print("")


In [ ]:
df.shape

In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
from numpy.random import seed
seed(1337)
from keras import optimizers
labels = df["Sarcasm"]
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 50, input_length=50, weights=[embedding_matrix], trainable=True))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))


model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))


sgd = keras.optimizers.SGD(learning_rate=0.001)

model_glove.compile(loss='binary_crossentropy', optimizer=sgd, metrics=[f1_m ,'acc'] )
## Fit train data

history = model_glove.fit(data, np.array(labels), validation_split=0.1, epochs = 20  )

In [ ]:
from keras.utils import plot_model
plot_model(model_glove, to_file='model.png')

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
NN = model_glove

In [ ]:
senti = pd.DataFrame()
senti['pos'] = posi
senti['neg'] = negi
senti['neu'] = neuu
senti['compund'] = compi

In [ ]:
X_train, X_test, y_train, y_test = tts(senti, labels, test_size=0.2,  random_state=13)

In [ ]:
rfc = RFC(n_jobs= -1)
rfc.fit(X_train, y_train)

In [ ]:
lr = LR()
lr.fit(X_train,y_train)

In [ ]:
huf_xgb = xgb.XGBClassifier(n_estimators = 500)
huf_xgb.fit(X_train,y_train)

In [ ]:
#CROSS VALIDATION 
##cross_val_score(model, X, y , cv = 5)
#model = model you want to validate 
#X - feature data 
#y - labels
#cv = number of folds you want in cross validation
# scores = cross_val_score(model, data, labels, cv=5,scoring ="f1")
# print("SCORES : " ,scores , "\nMEAN :" , scores.mean())

In [ ]:
models = [lr,huf_xgb ,rfc] 
f1score =[]
auc_score = []
accuracy = []
for i in models :
    pred = np.round(i.predict(X_test)).astype(int)
    f1score.append(f1_score(pred,y_test))
    auc_score.append(roc_auc_score(pred,y_test))
    accuracy.append(accuracy_score(pred,y_test))
    
        

In [ ]:
results = pd.DataFrame()
results["MODEL"] =[  "Logistic Regression","XGB","rfc"]
results["F1_Score"] =f1score
results["Auc_Score"] =auc_score
results["Accuracy %"]=accuracy

In [ ]:
results

In [ ]:
nn_pred = np.round(NN.predict(X_test))
lr_pred = huf_xgb.predict(X_test)
new_data = pd.DataFrame()
nn_pred.reshape(-1,)
new_data["NN"] = (nn_pred.reshape(-1,))
new_data["LR"] = lr_pred
new_data["NN"] = new_data["NN"].astype(int)
meta_X_train, meta_X_test, meta_y_train, meta_y_test = tts(new_data, y_test, test_size=0.2,  random_state=13)
from sklearn.naive_bayes import BernoulliNB as bayes

meta = xgb.XGBClassifier(n_estimators = 500)
meta.fit(meta_X_train,meta_y_train)
pred = np.round(meta.predict(meta_X_test)).astype(int)
meta_result = (f1_score(pred,meta_y_test))
print("STACKING RESULT :" ,  meta_result)

In [ ]:
average = []
for i in range(0 ,len(nn_pred)):
       average.append((0.89*nn_pred[i] + 0.90*lr_pred[i]) /(0.89+0.90))

In [ ]:
print("AVERAGE RESULT : " , f1_score(np.round(average) , y_test))